![saberpro.png](saberpro.png)

# Modelado - Proyecto Visualización y Storytelling

#### Librerías a importar
A continuación, se muestran las librerías a importar para poder implementar los procedimientos de este notebook:

In [1]:
# Paquetes necesarios
import pandas as pd
import numpy as np
from zipfile import ZipFile #para descomprimir los archivos
import os #para usar la funcion listdir() y crear una lista del directorio
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import table
from datetime import datetime, date

#### Importar la base de datos
Se importa el archivo de datos a la sesión de trabajo.

In [2]:
## Datos icfes - clasificacion de planteles
### https://icfesgovco-my.sharepoint.com/personal/dataicfes_icfes_gov_co/_layouts/15/onedrive.aspx?ct=1589295914045&or=OWA%2DNT&cid=27943174%2D8b52%2Deb60%2Dc2bf%2Dfd8870b02519&ga=1
### contraseña: Icfes2020*

RUTA = r"Datos"
ARCHIVOS = os.listdir(RUTA) ## lista donde se guardaran los nombres de los archivos con las bases de datos
# len(ARCHIVOS)#, ARCHIVOS #numero de archivos en formato zip

## Funcion para Extraer los archivos de cada zip a una carpeta temporal
def extraer_a_temporal(ARCHIVOS, RUTA):
    # Para cada zip que se encuentra en la lista de ARCHIVOS
    for archivo in ARCHIVOS:
        # Lee el contenido de cada zip y crea una lista de los archivos que contiene
        with ZipFile(RUTA+"/"+archivo, 'r') as archivo_2:
            name_archivo = archivo_2.namelist()
            # Lee cada archivo dentro del zip y si termina en .txt entonces lo extrae en la carpeta BASES
            for nombre in name_archivo:
                if nombre.endswith('.txt') or nombre.endswith('.TXT') or nombre.endswith('.csv'):
                    archivo_2.extract(nombre, 'BASES')
    return

def importar(RUTA_2,SEPARADOR,name_archivos, enc = 'Windows-1252'):
#     name_archivos = os.listdir(RUTA_2)
    lista_archivos = []
    for name in name_archivos:
        if name.endswith('.txt') or name.endswith('.TXT') or name.endswith('.csv'):
            data = pd.read_csv(RUTA_2+'/'+name, sep = SEPARADOR, engine = 'python',encoding = enc)
            data["Fuente"] = name
            lista_archivos.append(data)
        
    icfesDF = pd.concat(lista_archivos, ignore_index = True)
    return icfesDF

In [3]:
## Ejecutar la función para extraer los archivos a una carpeta temporal
extraer_a_temporal(ARCHIVOS, RUTA)

In [4]:
RUTA_2 = os.path.abspath(os.getcwd())+'\\'+'BASES'
# os.listdir(RUTA_2)
name_archivos_2 = ['SB11-CLASIFI-PLANTELES-20182.txt', 'SB11-CLASIFI-PLANTELES-20191.txt',
                   'SB11-CLASIFI-PLANTELES-20194.txt', 'SB11-CLASIFI-PLANTELES-20201.txt',
                   'SB11-CLASIFI-PLANTELES-20204.txt', 'SB11-CLASIFI-PLANTELES-20211.txt'] # sep = '¬' '|',encoding = 'utf-8'
name_archivos_3 = ['SB11-CLASIFI-PLANTELES-20171.csv','SB11-CLASIFI-PLANTELES-20172.csv'] # sep = '¬' '|',encoding = 'utf-8

## Importar 20181
icfes_1 = pd.read_csv(RUTA_2+'/'+'SB11-CLASIFI-PLANTELES-20181.txt', sep = '\t',engine = 'python', encoding = 'Windows-1252')
icfes_1 = icfes_1.loc[:, ~icfes_1.columns.str.startswith('|')]
icfes_1["Fuente"] = 'SB11-CLASIFI-PLANTELES-20181.txt'

# Importar archivos con caracteristicas similares de sep y encoding
SEPARADOR_2 = '[|¬]'
icfes_2 = importar(RUTA_2,SEPARADOR_2,name_archivos_2, enc = 'utf-8')

# Importar archivos con caracteristicas similares de sep y encoding
SEPARADOR_3 = ','
icfes_3 = importar(RUTA_2,SEPARADOR_3,name_archivos_3, enc = 'utf-8')

## Crear un solo dataframe
icfes = pd.concat([icfes_1, icfes_2, icfes_3], ignore_index = True)
len(icfes["Fuente"].unique()), len(icfes.columns)

(9, 21)

Se cargaron 9 archivos de la base de clasificación de planteles del ICFES de los años 2017-2021; para cada año se cargaron dos periodos que corresponden a las dos fechas en las que se desarrolla el examen del icfes durante cada año, a excepción del año 2021 para el que se tiene solo 1 periodo.

In [5]:
## Crear un data frame con la información de los periodos a partir de los nombres de los archivos
# icfes["Fuente"].unique() #Nombres de los archivos
Orden_fuente = [['SB11-CLASIFI-PLANTELES-20171.csv', 2017, 1, 1],
                ['SB11-CLASIFI-PLANTELES-20172.csv', 2017, 2, 2],
                ['SB11-CLASIFI-PLANTELES-20181.txt', 2018, 1, 3],
                ['SB11-CLASIFI-PLANTELES-20182.txt', 2018, 2, 4],
                ['SB11-CLASIFI-PLANTELES-20191.txt', 2019, 1, 5],
                ['SB11-CLASIFI-PLANTELES-20194.txt', 2019, 2, 6],
                ['SB11-CLASIFI-PLANTELES-20201.txt', 2020, 1, 7],
                ['SB11-CLASIFI-PLANTELES-20204.txt', 2020, 2, 8],
                ['SB11-CLASIFI-PLANTELES-20211.txt', 2021, 1, 9]]

fuenteDF = pd.DataFrame(Orden_fuente, columns = ['Fuente', 'Anio', 'Periodo', 'Sort_global'])
# fuenteDF

## Se unen la BD con el nuevo DF para una mejor identificacion de los registros segun el periodo
icfes = icfes.merge(fuenteDF, how = "left", on = "Fuente")

print(f"La base de datos tiene {icfes.shape[0]} registros y {icfes.shape[1]} columnas/variables, \
que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo', 'Anio', 'Sort_global'.")

print(f"Además, tenemos información de 5 años y 9 periodos. \n \
\n {icfes.groupby(by = ['Anio', 'Periodo']).size()}")

La base de datos tiene 49191 registros y 24 columnas/variables, que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo', 'Anio', 'Sort_global'.
Además, tenemos información de 5 años y 9 periodos. 
 
 Anio  Periodo
2017  1            291
      2           9071
2018  1            281
      2           9150
2019  1            335
      2           8846
2020  1            305
      2           8851
2021  1          12061
dtype: int64


In [6]:
## Obtener la base de datos con las columnas de interes
# list(icfes.columns.values)
lista_columnas = ['COLE_COD_DANE','COLE_INST_NOMBRE', 'COLE_CODMPIO_COLEGIO', 'COLE_MPIO_MUNICIPIO',
                  'COLE_COD_DEPTO',  'COLE_DEPTO_COLEGIO', 'COLE_NATURALEZA', 
                  'COLE_CALENDARIO_COLEGIO', 'COLE_CATEGORIA', 'Anio', 'Periodo', 'Sort_global']
icfes_final = icfes.loc[:, lista_columnas]

print(f"Después de quedarnos solo con la información de interes, la nueva base de \
datos tiene {icfes_final.shape[0]} registros (se eliminaron {icfes.shape[0] - icfes_final.shape[0]} registros) y {icfes_final.shape[1]} columnas/variables, \
que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo', 'Anio', 'Sort_global'.")
icfes_final.head()

Después de quedarnos solo con la información de interes, la nueva base de datos tiene 49191 registros (se eliminaron 0 registros) y 12 columnas/variables, que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo', 'Anio', 'Sort_global'.


,COLE_COD_DANE,COLE_INST_NOMBRE,COLE_CODMPIO_COLEGIO,COLE_MPIO_MUNICIPIO,COLE_COD_DEPTO,COLE_DEPTO_COLEGIO,COLE_NATURALEZA,COLE_CALENDARIO_COLEGIO,COLE_CATEGORIA,Anio,Periodo,Sort_global
0,3.080011e+11,AMERICAN SCHOOL,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
1,3.085730e+11,INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
2,3.080011e+11,COLEGIO ALTAMIRA,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
3,3.080011e+11,COLEGIO BRITANICO INTERNACIONAL,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
4,3.085731e+11,COLEGIO SAN JOSÉ,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3


#### Limpieza de los campos y análisis descriptivos de las variables

A continuación, se verifica si las columnas tienen valores faltantes. Además, se transforma la columna del código DANE de la base de datos para tenerla en formato string. Se renombran las columnas de la base para que sea más sencillo el proceso de modelado. Finalmente, se estandarizan los nombres de los municipios, de los departamentos y de los colegios.

In [7]:
## Verificando los valores missing
## Creando el porcentaje de valores missing de toda la base
null_columnas = icfes_final.columns[icfes_final.isnull().any()]
name_columnas_null = icfes_final[null_columnas].isnull().sum() *100 / icfes_final.shape[0]
name_columnas_null = name_columnas_null.sort_values(ascending = False)

print(f"En general, no hay variables ({len(name_columnas_null)} de las {icfes_final.shape[1]}) con missing values.")
name_columnas_null

En general, no hay variables (0 de las 12) con missing values.


Series([], dtype: float64)

In [8]:
## Tipos de variables en la base de datos (hasta este momento la BD mantiene los mismos indices desde su importación)
icfes_final[['COLE_COD_DANE','Otro']] = icfes_final["COLE_COD_DANE"].astype('str').str.split(".",expand=True)
icfes_final.drop(['Otro'], axis=1, inplace=True)
icfes_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49191 entries, 0 to 49190
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   COLE_COD_DANE            49191 non-null  object
 1   COLE_INST_NOMBRE         49191 non-null  object
 2   COLE_CODMPIO_COLEGIO     49191 non-null  int64 
 3   COLE_MPIO_MUNICIPIO      49191 non-null  object
 4   COLE_COD_DEPTO           49191 non-null  int64 
 5   COLE_DEPTO_COLEGIO       49191 non-null  object
 6   COLE_NATURALEZA          49191 non-null  object
 7   COLE_CALENDARIO_COLEGIO  49191 non-null  object
 8   COLE_CATEGORIA           49191 non-null  object
 9   Anio                     49191 non-null  int64 
 10  Periodo                  49191 non-null  int64 
 11  Sort_global              49191 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 4.9+ MB


Ahora, se realizan las estadísticas descriptivas de los puntajes de los estudiantes.

In [9]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) #modificando el formato de visualizacion del describe

## Estadísticas descriptivas de los puntajes de los estudiantes
print(f"A continuación se presenta para los puntajes del estudiante, el número de observaciones, \
la media, la desviación estandar, el minimo y el maximo, y los percentiles.")

icfes_final.describe().transpose()

A continuación se presenta para los puntajes del estudiante, el número de observaciones, la media, la desviación estandar, el minimo y el maximo, y los percentiles.


,count,mean,std,min,25%,50%,75%,max
COLE_CODMPIO_COLEGIO,49191.00,35192.59,26658.89,5001.00,11001.00,25126.00,63272.00,99773.00
COLE_COD_DEPTO,49191.00,34.91,26.64,5.00,11.00,25.00,63.00,99.00
Anio,49191.00,2019.10,1.45,2017.00,2018.00,2019.00,2020.00,2021.00
Periodo,49191.00,1.73,0.44,1.00,1.00,2.00,2.00,2.00
Sort_global,49191.00,5.94,2.61,1.00,4.00,6.00,8.00,9.00


In [10]:
## Estadísticas descriptivas de los estudiantes
print(f"A continuación se presenta para las variables categoricas del estudiante, el número de observaciones, \
los dominios, la moda y el número de veces que se repite la moda en la base de datos.")
icfes_final.describe(include='object').transpose()

A continuación se presenta para las variables categoricas del estudiante, el número de observaciones, los dominios, la moda y el número de veces que se repite la moda en la base de datos.


,count,unique,top,freq
COLE_COD_DANE,49191,12985,311001000000,537
COLE_INST_NOMBRE,49191,12433,INSTITUCION EDUCATIVA SIMON BOLIVAR,83
COLE_MPIO_MUNICIPIO,49191,1409,BOGOTÁ D.C.,5433
COLE_DEPTO_COLEGIO,49191,34,ANTIOQUIA,5565
COLE_NATURALEZA,49191,2,OFICIAL,34177
COLE_CALENDARIO_COLEGIO,49191,3,A,47449
COLE_CATEGORIA,49191,5,D,14370


In [11]:
icfes_final.groupby(['COLE_CALENDARIO_COLEGIO'])['COLE_CALENDARIO_COLEGIO'].count()

COLE_CALENDARIO_COLEGIO
A    47449
B     1676
O       66
Name: COLE_CALENDARIO_COLEGIO, dtype: int64

Solo vamos a seleccionar los colegios con calendario A y B, así que prescindiremos de los colegios de calendario O (flexible u otro).

In [12]:
icfes_final = icfes_final[icfes_final['COLE_CALENDARIO_COLEGIO'] != 'O']
icfes_final.shape

(49125, 12)

Ahora se modifican los nombres de las variables para que sean faciles de usar en la visualización

In [13]:
# Diccionario con los nombres de las variables (old) y los nuevos nombres
# list(icfes_final.columns)
names_old_new = {'COLE_INST_NOMBRE' : 'Nombre_Colegio',
                 'COLE_CODMPIO_COLEGIO' : 'Cod_Municipio',
                 'COLE_MPIO_MUNICIPIO' : 'Municipio',
                 'COLE_COD_DEPTO' : 'Cod_Departamento',
                 'COLE_DEPTO_COLEGIO' : 'Departamento',
                 'COLE_NATURALEZA' : 'Tipo',
                 'COLE_CALENDARIO_COLEGIO' : 'Calendario',
                 'COLE_CATEGORIA' : 'Clasificacion',
                 'Anio' : 'Anio_prueba',
                 'Periodo' : 'Periodo_prueba',
                 'Sort_global' : 'Orden_prueba',
                 }
icfes_final = icfes_final.rename(columns = names_old_new, inplace=False)
icfes_final.head()

,COLE_COD_DANE,Nombre_Colegio,Cod_Municipio,Municipio,Cod_Departamento,Departamento,Tipo,Calendario,Clasificacion,Anio_prueba,Periodo_prueba,Orden_prueba
0,308001074789,AMERICAN SCHOOL,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
1,308573000450,INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
2,308001073952,COLEGIO ALTAMIRA,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
3,308001101153,COLEGIO BRITANICO INTERNACIONAL,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3
4,308573074909,COLEGIO SAN JOSÉ,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3


Verificar los nombres de los municipios, de los departamentos y de los colegios que requieren algún tipo de estandarización.

In [14]:
len(icfes_final['Cod_Municipio'].unique()) ,len(icfes_final['Municipio'].unique())

(1116, 1409)

In [15]:
len(icfes_final['Cod_Departamento'].unique()), len(icfes_final['Departamento'].unique())

(33, 34)

In [16]:
len(icfes_final['COLE_COD_DANE'].unique()), len(icfes_final['Nombre_Colegio'].unique())

(12945, 12402)

In [17]:
## Se tienen dos codigos de municipio que no corresponden
# icfes_final[(icfes_final['Cod_Municipio'] == 27086) | (icfes_final['Cod_Municipio'] == 94663)]
icfes_final['Cod_Municipio'] = icfes_final['Cod_Municipio'].replace(94663, 94343)
icfes_final['Cod_Municipio'] = icfes_final['Cod_Municipio'].replace(27086, 27615)
# 94663 = 94343
# 27086 = 27615

In [18]:
## Seleccionar el codigo y el nombre del colegio para unificar y eliminar los duplicados
icfes_code_colegio = icfes_final[['COLE_COD_DANE','Nombre_Colegio']]
icfes_code_colegio.drop_duplicates(subset=['COLE_COD_DANE'], inplace=True, ignore_index = True)

## Eliminar la variable nombre colegio y hacer el merge con los codigos sin duplicados
icfes_final = icfes_final.drop(['Nombre_Colegio'], axis = 1)
df_final = icfes_final.merge(icfes_code_colegio, how = 'left', on = 'COLE_COD_DANE')
df_final.head()

C:\Users\jesus\AppData\Local\Temp\ipykernel_20004\3919980043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icfes_code_colegio.drop_duplicates(subset=['COLE_COD_DANE'], inplace=True, ignore_index = True)


,COLE_COD_DANE,Cod_Municipio,Municipio,Cod_Departamento,Departamento,Tipo,Calendario,Clasificacion,Anio_prueba,Periodo_prueba,Orden_prueba,Nombre_Colegio
0,308001074789,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3,AMERICAN SCHOOL
1,308573000450,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3,INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...
2,308001073952,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3,COLEGIO ALTAMIRA
3,308001101153,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3,COLEGIO BRITANICO INTERNACIONAL
4,308573074909,8573,PUERTO COLOMBIA,8,ATLANTICO,NO OFICIAL,B,A+,2018,1,3,COLEGIO SAN JOSÉ


In [19]:
len(df_final['COLE_COD_DANE'].unique()), len(df_final['Nombre_Colegio'].unique())

(12945, 9936)

Teniendo en cuenta que no se encuentran estandarizados los municipios y departamentos, se procede a estandarizar con base en los codigos divipola del DANE.

In [20]:
## Cargar dataframe con el que se va a solucionar
df_base = pd.read_excel('Intermedia/Listados_DIVIPOLA.xlsx', sheet_name = 'Municipios', skiprows = 10, nrows = 1121)
names_old_new = {'Código .1' : 'Cod_Municipio',
                 'Nombre.1' : 'Municipio',
                 'Nombre' : 'Departamento'
                }
df_base = df_base.rename(columns = names_old_new, inplace=False)

## Seleccionar columnas de interes
lista_columnas = ['Cod_Municipio','Municipio', 'Departamento']
df_base = df_base.loc[:, lista_columnas]
df_base

,Cod_Municipio,Municipio,Departamento
0,5001,MEDELLÍN,ANTIOQUIA
1,5002,ABEJORRAL,ANTIOQUIA
2,5004,ABRIAQUÍ,ANTIOQUIA
3,5021,ALEJANDRÍA,ANTIOQUIA
4,5030,AMAGÁ,ANTIOQUIA
...,...,...,...
1116,97889,YAVARATÉ,VAUPÉS
1117,99001,PUERTO CARREÑO,VICHADA
1118,99524,LA PRIMAVERA,VICHADA
1119,99624,SANTA ROSALÍA,VICHADA


In [21]:
## Eliminar la variable municipio y departamento y hacer el merge con los codigos sin duplicados
df_final = df_final.drop(['Municipio','Departamento'], axis = 1)
df_final = df_final.merge(df_base, how = 'left', on = 'Cod_Municipio')
df_final

,COLE_COD_DANE,Cod_Municipio,Cod_Departamento,Tipo,Calendario,Clasificacion,Anio_prueba,Periodo_prueba,Orden_prueba,Nombre_Colegio,Municipio,Departamento
0,308001074789,8573,8,NO OFICIAL,B,A+,2018,1,3,AMERICAN SCHOOL,PUERTO COLOMBIA,ATLÁNTICO
1,308573000450,8573,8,NO OFICIAL,B,A+,2018,1,3,INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...,PUERTO COLOMBIA,ATLÁNTICO
2,308001073952,8573,8,NO OFICIAL,B,A+,2018,1,3,COLEGIO ALTAMIRA,PUERTO COLOMBIA,ATLÁNTICO
3,308001101153,8573,8,NO OFICIAL,B,A+,2018,1,3,COLEGIO BRITANICO INTERNACIONAL,PUERTO COLOMBIA,ATLÁNTICO
4,308573074909,8573,8,NO OFICIAL,B,A+,2018,1,3,COLEGIO SAN JOSÉ,PUERTO COLOMBIA,ATLÁNTICO
...,...,...,...,...,...,...,...,...,...,...,...,...
49120,223189000000,23189,23,OFICIAL,A,D,2017,2,2,I.E. SAN JOSE DE LA GUNETA,CIÉNAGA DE ORO,CÓRDOBA
49121,311001000000,11001,11,NO OFICIAL,A,A,2017,2,2,COL BILING RICHMOND,"BOGOTÁ, D.C.","BOGOTÁ, D.C."
49122,320250000000,20250,20,NO OFICIAL,A,D,2017,2,2,COLEGIO NUESTRA SEÑORA DE FATIMA,EL PASO,CESAR
49123,320001000000,20001,20,NO OFICIAL,A,D,2017,2,2,COLEGIO SAN ANTONIO,VALLEDUPAR,CESAR


In [22]:
# Verificar si hay nulos
# df_final[pd.isnull(df_final['Departamento'])]
len(df_final['Cod_Departamento'].unique()), len(df_final['Departamento'].unique())

(33, 33)

In [23]:
len(df_final['Cod_Municipio'].unique()) ,len(df_final['Municipio'].unique())

(1114, 1031)

In [24]:
df_final = df_final.drop(['COLE_COD_DANE','Cod_Municipio','Cod_Departamento','Periodo_prueba','Orden_prueba'], axis=1)
df_final.to_csv('Base_final_icfes.csv')
df_final

,Tipo,Calendario,Clasificacion,Anio_prueba,Nombre_Colegio,Municipio,Departamento
0,NO OFICIAL,B,A+,2018,AMERICAN SCHOOL,PUERTO COLOMBIA,ATLÁNTICO
1,NO OFICIAL,B,A+,2018,INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...,PUERTO COLOMBIA,ATLÁNTICO
2,NO OFICIAL,B,A+,2018,COLEGIO ALTAMIRA,PUERTO COLOMBIA,ATLÁNTICO
3,NO OFICIAL,B,A+,2018,COLEGIO BRITANICO INTERNACIONAL,PUERTO COLOMBIA,ATLÁNTICO
4,NO OFICIAL,B,A+,2018,COLEGIO SAN JOSÉ,PUERTO COLOMBIA,ATLÁNTICO
...,...,...,...,...,...,...,...
49120,OFICIAL,A,D,2017,I.E. SAN JOSE DE LA GUNETA,CIÉNAGA DE ORO,CÓRDOBA
49121,NO OFICIAL,A,A,2017,COL BILING RICHMOND,"BOGOTÁ, D.C.","BOGOTÁ, D.C."
49122,NO OFICIAL,A,D,2017,COLEGIO NUESTRA SEÑORA DE FATIMA,EL PASO,CESAR
49123,NO OFICIAL,A,D,2017,COLEGIO SAN ANTONIO,VALLEDUPAR,CESAR
